In [1]:
from modeling.couplingapi import Subproblem
from modeling.api import Var
import openmdao.api as om
from modeling.ipoptsolver import setup_ipopt
import numpy as np
import jax.numpy as anp

### Explicit equations *with* coupling

In [21]:
P1 = Subproblem()
x3 = Var('x_3')
a = Var('a')
x1 = P1.add_equation('x_1', x3**2-a)
x2 = P1.add_equation('x_2', x1*x3)
P1.add_equation(x3, x1*x2)
P1.variable_order = ['a','x_1','x_2','x_3']
y = np.array([2, -1, -1, 1], dtype=np.float32)
P1.solve(y)

array([ 2., -1., -1.,  1.], dtype=float32)

In [22]:
x1.shape = (1,)
x2.shape = (1,)
x3.shape = (1,)
a.shape = (1,)
Pipopt, indep_order = setup_ipopt(P1.components, [a, x1, x2, x3], y)

In [23]:
indep_order

[x_3, x_1, x_2]

In [24]:
Pipopt.constraints(anp.array([1,-1,-1], dtype=np.float32))

DeviceArray([0., 0., 0.], dtype=float32)

In [26]:
Pipopt.jacobian(anp.array([1,-1,-1], dtype=np.float32))

DeviceArray([[-2.,  0.,  1.],
             [ 1.,  1., -1.],
             [ 1.,  1.,  1.]], dtype=float32)

### Explicit equations without coupling

In [7]:
P1 = Subproblem()
x3 = Var('x_3')
a = Var('a')
x1 = P1.add_equation('x_1', x3**2-a)
x2 = P1.add_equation('x_2', x1*x3)
P1.variable_order = ['a','x_1','x_2','x_3']
y = np.array([1, 0, 0, 1], dtype=np.float32)
P1.solve(y)

array([1., 0., 0., 1.], dtype=float32)

### Implicit/residual equations

In [18]:
P1 = Subproblem()
x = Var('x')
b = Var('b')
a = Var('a')
x1 = P1.add_equation(x**2+a*x+b)
P1.projected = ['a','b']
P1.projection_order = ['a','b','x']

x_3

## Merge

In [ ]:
MA = M1.elminate(M2)
MB = merge_solvers(M1, M2)

In [30]:
import numpy as np
from modeling.execution import Component
A = np.random.rand(10,10)
b0 = np.random.rand(10)
x0 = np.random.rand(10) 
c = 1
fx = lambda x,b: (b@(A@x-b),) # The @ is numpys matrix multiplication
component = Component(fx, inputs=('x','b'), outputs=('y',), indims=(10,10), outdims=(1,))
component2 = Component(lambda x,c: (sum(x),x[0]-c), inputs=('x','c'), outputs=('z',), indims=(10,1), outdims=(2,))

In [31]:
components = (component, component2)
outdims_merge = 0
inps = dict()
for c in components:
    inps.update({inp: c.indims[idx] for idx, inp in enumerate(c.inputs)})
    outdims_merge+= sum(c.outdims)
indims_merge = sum(inps.values())

In [ ]:
from modeling.couplingapi import find_indices

In [32]:
idxs = [find_indices(P1.projection_order, 
                             c.inputs) for c in components]
output = []
for idx, c in enumerate(self.components):
    xidxs = x[idxs[idx]]
    fxval = c.function(*xidxs)
    outval = x[sum(c.indims):]
    out = [outval[idx]-elt for idx,elt in enumerate(fxval)]
    output.extend([elt for vector in out for elt in vector])

3

In [33]:
indims_merge

21

In [3]:
P1.projection_order = ['a','x_1','x_2','x_3']

In [4]:
P1.projected

{'a', 'x_1', 'x_2', 'x_3'}

In [5]:
y = [3,-1,2**0.5,-2**0.5]
P1.solve(y)

array([ 3.        , -1.        ,  1.41421356, -1.41421356])

In [6]:
y = [3, -3,  0,  0]
P1.solve(y)

array([ 3, -3,  0,  0])

In [7]:
P1.Vtree = {'x_1':1, 'x_2':1, 'x_3':1}

In [8]:
P1.setup()

In [9]:
P1.prob.model.s1.linear_solver = om.DirectSolver()
newton = P1.prob.model.s1.nonlinear_solver #= om.NewtonSolver()
newton.linesearch = om.BoundsEnforceLS()
newton.linesearch.options['iprint'] = 2
#P1.prob.model.s1.nonlinear_solver = om.NonlinearRunOnce()
P1.prob.setup()

In [10]:
yi = [3, -1, 1.2,-1.2]

In [11]:
yi = P1.solve_with_engine(yi)
yi


==
s1
==
|  LS: BCHK 0 ; 1.3928781 0.183773963
|  LS: BCHK 0 ; 3.53448385 1.11929885
|  LS: BCHK 0 ; 4.26067927 0.217249187
|  LS: BCHK 0 ; 55.4870334 0.245470305
|  LS: BCHK 0 ; 1000422.45 0.00163147941
|  LS: BCHK 0 ; 1733781.13 1.73380193e-24
|  LS: BCHK 0 ; 1733781.13 1.73380193e-24
|  LS: BCHK 0 ; 1733781.13 1.73380193e-24
|  LS: BCHK 0 ; 1733781.13 1.73380193e-24
|  LS: BCHK 0 ; 1733781.13 1.73380193e-24
NL: NewtonSolver 'NL: Newton' on system 's1' failed to converge in 10 iterations.


array([3.00000000e+00, 0.00000000e+00, 9.99948288e+08, 9.99994000e+14])

In [14]:
P1.prob.model.list_inputs()

6 Input(s) in 'model'

varname  val  
-------  -----
s1
  f0
    x_3  [0.] 
    a    [3.] 
  f1
    x_1  [-3.]
    x_3  [0.] 
  f2
    x_1  [-3.]
    x_2  [-0.]




[('s1.f0.x_3', {'val': array([0.])}),
 ('s1.f1.x_3', {'val': array([0.])}),
 ('s1.f0.a', {'val': array([3.])}),
 ('s1.f1.x_1', {'val': array([-3.])}),
 ('s1.f2.x_1', {'val': array([-3.])}),
 ('s1.f2.x_2', {'val': array([-0.])})]

In [9]:
for elt in P1.mdao_in:
    idx = P1.projection_order.index(elt)
    P1.prob.set_val(elt, y[idx])

In [17]:
P1.prob['a']=3
P1.prob['x_1']=0
P1.prob['x_2']=0
P1.prob['x_3']=0

In [18]:
P1.prob.run_model()

In [57]:
P1.prob.model.s1.f2._inputs

In [46]:
P1.prob.model.s1.f1._var_rel2meta['x_1']

{'val': array([1000]),
 'shape': (1,),
 'size': 1,
 'src_indices': None,
 'flat_src_indices': None,
 'add_input_src_indices': False,
 'units': None,
 'desc': '',
 'distributed': False,
 'tags': set(),
 'shape_by_conn': False,
 'copy_shape': None,
 'global_shape': (1,),
 'global_size': 1}

In [38]:
P1.prob.model.s1.f2._var_rel2meta

{'x_3': {'val': array([1.]),
  'shape': (1,),
  'size': 1,
  'units': None,
  'res_units': None,
  'desc': '',
  'distributed': False,
  'tags': set(),
  'ref': 1.0,
  'ref0': 0.0,
  'res_ref': 1.0,
  'lower': None,
  'upper': None,
  'shape_by_conn': False,
  'copy_shape': None,
  'global_shape': (1,),
  'global_size': 1},
 'x_1': {'val': array([1.]),
  'shape': (1,),
  'size': 1,
  'src_indices': None,
  'flat_src_indices': None,
  'add_input_src_indices': False,
  'units': None,
  'desc': '',
  'distributed': False,
  'tags': set(),
  'shape_by_conn': False,
  'copy_shape': None,
  'global_shape': (1,),
  'global_size': 1},
 'x_2': {'val': array([1.]),
  'shape': (1,),
  'size': 1,
  'src_indices': None,
  'flat_src_indices': None,
  'add_input_src_indices': False,
  'units': None,
  'desc': '',
  'distributed': False,
  'tags': set(),
  'shape_by_conn': False,
  'copy_shape': None,
  'global_shape': (1,),
  'global_size': 1}}

In [10]:
P1.solve_with_engine([3,-1,2**0.5,-2**0.5])


==
s1
==
NL: NLBGSSolver 'NL: NLBGS' on system 's1': residuals contain 'inf' or 'NaN' after 6 iterations.


c:\Users\johan\miniconda3\envs\minimdo\lib\site-packages\openmdao\solvers\nonlinear\nonlinear_block_gs.py:206: RuntimeWarning: invalid value encountered in subtract
  residuals.set_val(outputs.asarray() - outputs_n)


array([[ 3.],
       [inf],
       [inf],
       [inf]])

In [11]:
P1.prob.model.list_outputs()

4 Explicit Output(s) in 'model'

varname  val  
-------  -----
inp
  a      [3.] 
s1
  f0
    x_1  [nan]
  f1
    x_2  [nan]
  f2
    x_3  [nan]


0 Implicit Output(s) in 'model'




[('inp.a', {'val': array([3.])}),
 ('s1.f0.x_1', {'val': array([nan])}),
 ('s1.f1.x_2', {'val': array([nan])}),
 ('s1.f2.x_3', {'val': array([nan])})]

In [11]:
import sympy as sp

x1, x2, x3 = sp.symbols('x1 x2 x3')

# Define the equations
eq1 = sp.Eq(x1, x3**2-3)
eq2 = sp.Eq(x2, x1 * x3)
eq3 = sp.Eq(x3, x1 * x2)

# Solve the system of equations
solution = sp.solve((eq1, eq2, eq3), (x1, x2, x3), dict=True)
print(solution)


[{x1: -3, x2: 0, x3: 0}, {x1: -1, x2: -sqrt(2), x3: sqrt(2)}, {x1: -1, x2: sqrt(2), x3: -sqrt(2)}, {x1: 1, x2: -2, x3: -2}, {x1: 1, x2: 2, x3: 2}]
